In [ ]:
import os
import glob

from moviepy.editor import VideoFileClip, concatenate_videoclips, CompositeVideoClip, TextClip, ImageClip, afx
import cairosvg
import html
from common import get_videos


video_length = 20
muted_length = 5
intro_length = 2
cross_fade = 2
temp_dir = 'svg_temp'
width = 1080
height = 1920

def get_songs(year, start, end):
    videos = get_videos(year, start, end)
    for video in videos:
        if video['status'] != 'OK':
            print(f'Video {video["ranking"]} {video["artist"]} {video["title"]} not found: {video["status"]}')
    return videos

def make_missing():
    svg = f"""<?xml version="1.0" encoding="UTF-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="1080" height="1920">
    <text font-family="Russo One" font-size="70" fill="white" stroke="black" stroke-width="3" text-anchor="middle" x="50%" y="50%">
        <tspan x="50%" dy="1em">Ääniraitaa ei löytynyt :(</tspan>
    </text>
</svg>
    """
    filename = f"{temp_dir}/missing.png"
    cairosvg.svg2png(bytestring=svg.encode(), write_to=filename, output_width=width, output_height=height)
    return ImageClip(filename)

def make_muted():
    svg = f"""<?xml version="1.0" encoding="UTF-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="1080" height="1920">
    <text font-family="Russo One" font-size="60" fill="white" stroke="black" stroke-width="3" text-anchor="middle" x="50%" y="50%">
        <tspan x="50%" dy="1em">Ääni poistettu tekijänoikeussyistä</tspan>
    </text>
</svg>
    """
    filename = f"{temp_dir}/muted.png"
    cairosvg.svg2png(bytestring=svg.encode(), write_to=filename, output_width=width, output_height=height)
    return ImageClip(filename)

def make_header(year, start, end):
    svg = f"""<?xml version="1.0" encoding="UTF-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="{width}" height="{height}">
    <text font-family="Russo One" font-size="60" fill="white" stroke="black" stroke-width="3" text-anchor="middle" x="50%" y="5%">
        <tspan x="50%" dy="1em">Suomen top-lista</tspan>
        <tspan x="50%" dy="1.1em" font-weight="bold">{year}</tspan>
        <tspan x="50%" dy="1.1em">Sijat {start} - {end}</tspan>
    </text>
</svg>
    """
    filename = f"{temp_dir}/header_{year}_{start}_{end}.png"
    cairosvg.svg2png(bytestring=svg.encode(), write_to=filename, output_width=width, output_height=height)
    return ImageClip(filename)

def make_cover(year, start, end):
    svg = f"""<?xml version="1.0" encoding="UTF-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="1080" height="1920">
  <polygon points="0,240 1000,240 1040,280 40,280 40,820 0,780" style="fill:#1fd5ee;" />
  <polygon points="1040,280 1040,820 40,820 80,860 1080,860 1080,320" style="fill:#fc2b55;" />
  <rect width="1000" height="540" style="fill:black;" x="40" y="280" />
  <text font-family="Russo One" font-size="104" fill="white" stroke="black" stroke-width="3" text-anchor="middle" x="50%" y="420">
    <tspan x="50%">Suomen top-lista</tspan>
    <tspan x="50%" dy="1.3em" font-weight="bold">{year}</tspan>
    <tspan x="50%" dy="1.3em">Sijat {start} - {end}</tspan>
  </text>
</svg>
    """
    filename = f"{temp_dir}/cover_{year}_{start}_{end}.png"
    cairosvg.svg2png(bytestring=svg.encode(), write_to=filename, output_width=width, output_height=height)
    return ImageClip(filename)

def make_song_title(year, ranking, artist, title):
    try:
        artist = html.escape(artist)
        title = html.escape(title)
        artist_size = 60
        title_size = 60
        
        if(len(artist) > 25):
            artist_size = 50
        if(len(artist) > 35):
            artist_size = 40

        if(len(title) > 25):
            title_size = 50
        if(len(title) > 35):
            title_size = 40

        svg = f"""<?xml version="1.0" encoding="UTF-8"?>
    <svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="{width}" height="{height}">
        <text font-family="Russo One" font-size="60" fill="white" stroke="black" stroke-width="3" text-anchor="middle" x="50%" y="84%">
            <tspan x="50%" dy="1em" font-weight="bold">{ranking}.</tspan>
            <tspan x="50%" dy="1em" font-size="{artist_size}">{artist}</tspan>
            <tspan x="50%" dy="1.1em" font-size="{title_size}">{title}</tspan>
        </text>
    </svg> 
        """
        filename = f"{temp_dir}/{year}_{ranking}.png"
        cairosvg.svg2png(bytestring=svg.encode(), write_to=filename, output_width=width, output_height=height)
        return ImageClip(filename)
    except:
        print(f'Error with {ranking} {artist} {title}')
        raise

def build_video(year, start, end):
    
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    songs = get_songs(year, start, end)
    songs = sorted(songs, key=lambda k: k['ranking'], reverse=True)

    

    # intro
    intro_bg = (ImageClip('cover3_2.png')
                .set_position(("center","center"))
                .set_duration(intro_length)
                .resize(width=width, height=height))
    intro_cover = (make_cover(year, start, end)
                   .set_position(("center","center"))
                   .set_duration(intro_length))
    intro_video = (
        CompositeVideoClip([intro_bg, intro_cover], size=(width, height))
        .crossfadeout(1))

    clips = []

    for song in songs:

        length = video_length
        if song['audio_level'] == 0:
            length = muted_length

        print(f'Processing {song["ranking"]} {song["artist"]} {song["title"]} | {song["file"]} {song["start"]} -> {song["start"] + length}')

        if len(song['file']) == 0:
            video = make_missing().set_position(("center","center")).set_duration(3)
        else:
            video = (
                VideoFileClip(song['file'])
                .subclip(song['start'], song['start'] + length))
            video = video.set_position(("center","center"))
            video = video.audio_fadein(1).audio_fadeout(1)
            video = video.crossfadein(cross_fade)
            video = video.resize(height=1280)
            video = video.fx(afx.audio_normalize)
            if song['audio_level'] != 100:
                video = video.fx(afx.volumex, song['audio_level']/100)

        title = (
            make_song_title(year, song['ranking'], song['artist'], song['title'])
            .set_position(("center","bottom"))
            .set_duration(video.duration))
        title = title.crossfadein(1).crossfadeout(1)

        if song['audio_level'] == 0:
            muted = make_muted().set_position(("center","center")).set_duration(length)
            video = video.fx(afx.volumex, 0)
            clip = CompositeVideoClip([video, title, muted], size=(width, height))
        else:
            clip = CompositeVideoClip([video, title], size=(width, height))
    
        clips.append(clip)


    songs_video = concatenate_videoclips(clips, method="compose")

    header = make_header(year, start, end).set_position(("center","top")).set_duration(songs_video.duration)
    songs_headers_video = CompositeVideoClip([songs_video, header], size=(width, height))

    final_clip = concatenate_videoclips([intro_video, songs_headers_video], method="compose")

    final_clip.write_videofile(f'{year} {start}-{end}.mp4', fps=30)


    for f in glob.glob(f"{temp_dir}/*"):
        os.remove(f)





#build_video(1993, 46, 50)
#build_video(1993, 41, 45)
#build_video(1993, 36, 40)
#build_video(1993, 31, 35)
#build_video(1993, 26, 30)
build_video(1993, 21, 25)
#build_video(1993, 16, 20)
#build_video(1993, 11, 15)
#build_video(1993, 6, 10)
#build_video(1993, 1, 5)
"""
build_video(1991, 41, 50)
build_video(1991, 31, 40)
build_video(1991, 21, 30)
build_video(1991, 11, 20)
build_video(1991, 1, 10)
"""